
# Portfolio Optimization Overview

## Introduction
This notebook explores **portfolio optimization** techniques using financial data. We aim to construct an **efficient portfolio** that balances risk and return by optimizing asset weights.

## Data Collection
- We fetch stock price data from a financial source.
- Data is processed to compute **daily returns** and **covariance matrices** for risk assessment.

## Portfolio Returns & Risk Calculation
- Portfolio **expected return** is calculated as the weighted sum of asset returns.
- **Portfolio volatility (risk)** is derived from the covariance matrix.

## Efficient Frontier & Optimization
- The **Efficient Frontier** represents the set of portfolios offering the best return for a given level of risk.
- We use **Mean-Variance Optimization** to find the **minimum variance portfolio** and **maximum Sharpe ratio portfolio**.
- The optimization employs **SciPy’s `minimize` function** with constraints ensuring asset weights sum to 1.

## Results & Visualization
- **Plotly** is used to visualize the **Efficient Frontier**, showing optimal portfolios.
- The **Sharpe ratio maximization** technique helps identify the best risk-adjusted portfolio.


In [2]:
# Import warnings
import warnings
warnings.filterwarnings('ignore')

# Import pandas & yfinance
import pandas as pd
import numpy as np
from numpy.linalg import multi_dot #helps take scalar product of multiple vectors
import yfinance as yf

# Set numpy random seed
np.random.seed(42)

# Import plotly express for EF plot
import plotly.express as px
px.defaults.width, px.defaults.height = 1000, 600
import matplotlib.pyplot as plt
import cufflinks as cf
cf.set_config_file(offline=True)  # Enable offline mode         
# Set precision
pd.set_option('display.precision', 4)

In [3]:
#import and create database
from sqlalchemy import create_engine, text
engine = create_engine('sqlite:///Nifty50')

In [4]:
# Read data from wikipedia 
nifty50 = pd.read_html('https://en.wikipedia.org/wiki/NIFTY_50')[1].Symbol.to_list()
#  Fetch data from yahoo using list comprehension
data = [yf.download(symbol+'.NS', start="2020-01-27", end="2025-01-31", progress=False).reset_index() for symbol in nifty50] 


for frame, symbol in zip(data,nifty50):
    frame.to_sql(symbol,engine,if_exists ="replace",index= False)

frame.columns





YF.download() has changed argument auto_adjust default to True


MultiIndex([(  'Date',         ''),
            ( 'Close', 'WIPRO.NS'),
            (  'High', 'WIPRO.NS'),
            (   'Low', 'WIPRO.NS'),
            (  'Open', 'WIPRO.NS'),
            ('Volume', 'WIPRO.NS')],
           names=['Price', 'Ticker'])

In [5]:
# specify assets/ stocks
# Indian stocks: bank, consumer goods, diversified, tech, consumer durables

assets = sorted(['RELIANCE','TCS','ICICIBANK','NTPC','GRASIM'])

print(assets)

num_of_assets= len(assets)

num_of_portfolio = 5000

['GRASIM', 'ICICIBANK', 'NTPC', 'RELIANCE', 'TCS']


In [6]:
df = pd.DataFrame()

for asset in assets:
    # Fetch column names to identify correct ones
    with engine.connect() as connection:
        query_check = f'SELECT * FROM "{asset}" LIMIT 5'
        df_test = pd.read_sql_query(text(query_check), connection)
        actual_columns = df_test.columns

    # Fix Date Column Name
    date_col = [col for col in actual_columns if 'Date' in col]
    
    date_col = date_col[0]  # Extract actual column name

    # Fix Close Price Column Name
    close_col = [col for col in actual_columns if 'Close' in col and asset in col]
    
    close_col = close_col[0]  # Extract actual column name

    # Construct SQL query with correct column names
    query = f'SELECT "{date_col}", "{close_col}" FROM "{asset}"'

    with engine.connect() as connection:
        df1 = pd.read_sql_query(text(query), connection)

    # Rename Columns
    df1.columns = ["Date", asset]

    # Set Date as Index
    df1.set_index("Date", inplace=True)

    # Merge into main DataFrame
    df = pd.concat([df, df1], axis=1)

df.index= pd.to_datetime(df.index)
df


,GRASIM,ICICIBANK,NTPC,RELIANCE,TCS
Date,,,,,
2020-01-27,793.0153,519.3373,85.6421,674.2925,1989.7659
2020-01-28,788.9191,510.5891,84.8095,658.7170,2003.0659
2020-01-29,782.4821,509.0424,85.6800,662.3423,1976.3282
2020-01-30,771.7050,514.4557,86.0206,646.1849,1960.9640
2020-01-31,759.6600,508.1241,85.4150,631.8179,1907.0293
...,...,...,...,...,...
2025-01-24,2490.7500,1209.2000,321.1450,1246.3000,4152.3501
2025-01-27,2458.0500,1227.9500,319.7062,1229.3500,4065.1499
2025-01-28,2411.3000,1246.9000,315.7867,1234.4000,4040.3000


In [7]:
df.index

DatetimeIndex(['2020-01-27', '2020-01-28', '2020-01-29', '2020-01-30',
               '2020-01-31', '2020-02-03', '2020-02-04', '2020-02-05',
               '2020-02-06', '2020-02-07',
               ...
               '2025-01-17', '2025-01-20', '2025-01-21', '2025-01-22',
               '2025-01-23', '2025-01-24', '2025-01-27', '2025-01-28',
               '2025-01-29', '2025-01-30'],
              dtype='datetime64[ns]', name='Date', length=1242, freq=None)

In [9]:
df_normalised= df / df.iloc[0]
df_normalised.iplot(kind="line")

In [12]:
returns= df.pct_change()
annual_returns = round(returns.mean()*252*100,2)
annual_stdev= round(returns.std()*np.sqrt(252)*100,2)

df2 = pd.DataFrame({
    'Ann Ret':annual_returns,
    'Ann Std': annual_stdev

})

In [16]:
df2.reset_index().iplot(
    kind ="pie",
    labels= "index",
    values= "Ann Ret"
    
)

In [19]:
# portfolio simulations

In [28]:
def portfolio_simulation(returns):
    rets= [] ; vols= []; wts =[]

    for i in range(num_of_portfolio):
        weights= np.random.random(num_of_assets)
        weights /= np.sum(weights)
        rets.append (weights.T@np.array(returns.mean()*252))
        vols.append(np.sqrt(multi_dot([weights.T,returns.cov()*252,weights])*260))
        wts.append(weights)
    

    # create a dataframe for our analysis

    data= {'port_rets':rets, 'port_vols':vols}
    for counter , symbol in enumerate(returns.columns.tolist()):
        data[symbol +'_weight']= [w[counter] for w in wts]

    portdf = pd.DataFrame(data)
    portdf['sharpe_ratio']= portdf['port_rets']/portdf['port_vols']

    return round(portdf,4)

In [29]:
portfolios= portfolio_simulation(returns)
portfolios.head()

,port_rets,port_vols,GRASIM_weight,ICICIBANK_weight,NTPC_weight,RELIANCE_weight,TCS_weight,sharpe_ratio
0,0.2237,3.3651,0.1589,0.2404,0.1432,0.2378,0.2197,0.0665
1,0.2394,3.3767,0.0454,0.1210,0.3939,0.2656,0.1741,0.0709
2,0.2645,3.5123,0.1940,0.2063,0.4220,0.0480,0.1297,0.0753
3,0.2244,3.3271,0.1492,0.1938,0.1750,0.2613,0.2208,0.0674
4,0.2222,3.2588,0.2023,0.0780,0.1637,0.2518,0.3043,0.0682


In [34]:
portfolios.iloc[portfolios.sharpe_ratio.idxmax()]

port_rets           0.2614
port_vols           3.3772
GRASIM_weight       0.2070
ICICIBANK_weight    0.0948
NTPC_weight         0.4305
RELIANCE_weight     0.0093
TCS_weight          0.2584
sharpe_ratio        0.0774
Name: 4357, dtype: float64

In [36]:
portfolios.describe().T

,count,mean,std,min,25%,50%,75%,max
port_rets,5000.0,0.2338,0.0160,0.1789,0.2234,0.2338,0.2445,0.2902
port_vols,5000.0,3.4696,0.1756,3.1601,3.3383,3.4396,3.5770,4.3417
GRASIM_weight,5000.0,0.2007,0.1134,0.0000,0.1123,0.2010,0.2778,0.7630
ICICIBANK_weight,5000.0,0.1979,0.1135,0.0000,0.1084,0.1975,0.2733,0.6908
NTPC_weight,5000.0,0.2010,0.1140,0.0002,0.1142,0.1972,0.2777,0.7192
RELIANCE_weight,5000.0,0.2006,0.1154,0.0001,0.1092,0.2001,0.2798,0.6943
TCS_weight,5000.0,0.1997,0.1132,0.0001,0.1103,0.2006,0.2792,0.6754
sharpe_ratio,5000.0,0.0675,0.0049,0.0477,0.0642,0.0680,0.0712,0.0774


In [37]:
# Plot simulated portfolio
fig = px.scatter(
    portfolios, x='port_vols', y='port_rets', color='sharpe_ratio', 
    labels={'port_vols': 'Expected Volatility', 'port_rets': 'Expected Return','sharpe_ratio': 'Sharpe Ratio'},
    title="Monte Carlo Simulated Portfolio"
     ).update_traces(mode='markers', marker=dict(symbol='cross'))

# Plot max sharpe 
fig.add_scatter(
    mode='markers', 
    x=[portfolios.iloc[portfolios.sharpe_ratio.idxmax()]['port_vols']], 
    y=[portfolios.iloc[portfolios.sharpe_ratio.idxmax()]['port_rets']], 
    marker=dict(color='RoyalBlue', size=20, symbol='star'),
    name = 'Max Sharpe'
).update(layout_showlegend=False)

# Show spikes
fig.update_xaxes(showspikes=True)
fig.update_yaxes(showspikes=True)
fig.show()

In [38]:
# efficient frontier

In [39]:
import scipy.optimize as sco

In [40]:
def portfolio_stats(weights):
    
    weights = np.array(weights)
    port_rets = weights.T @ np.array(returns.mean() * 260)  
    port_vols = np.sqrt(multi_dot([weights.T, returns.cov() * 260, weights])) 
    
    return np.array([port_rets, port_vols, port_rets/port_vols])

# Minimize the volatility
def min_volatility(weights):
    return portfolio_stats(weights)[1]

# Minimize the variance
def min_variance(weights):
    return portfolio_stats(weights)[1]**2

# Maximizing sharpe ratio
def max_sharpe_ratio(weights):
    return -portfolio_stats(weights)[2]

In [41]:
# Specify constraints, bounds and initial weights
cons = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
bnds = tuple((0,1) for x in range(num_of_assets))
initial_wts = num_of_assets*[1./num_of_assets]

In [42]:
# Optimizing for maximum sharpe ratio
opt_sharpe = sco.minimize(max_sharpe_ratio, initial_wts, method='SLSQP', bounds=bnds, constraints=cons)

# Optimizing for minimum variance
opt_var = sco.minimize(min_variance, initial_wts, method='SLSQP', bounds=bnds, constraints=cons)

In [43]:
opt_sharpe

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -1.270000062918343
       x: [ 2.248e-01  5.831e-02  4.433e-01  4.710e-18  2.736e-01]
     nit: 5
     jac: [ 2.941e-04  5.668e-04  4.448e-04  9.096e-02 -1.088e-03]
    nfev: 31
    njev: 5

In [44]:
opt_var

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.039401518654493636
       x: [ 8.793e-02  8.580e-02  2.579e-01  1.289e-01  4.395e-01]
     nit: 7
     jac: [ 7.884e-02  7.878e-02  7.879e-02  7.879e-02  7.881e-02]
    nfev: 42
    njev: 7

In [51]:
# Efficient Frontier
targetrets = np.linspace(0.15,0.30,200)
tvols = []

for tr in targetrets:
    
    ef_cons = ({'type': 'eq', 'fun': lambda x: portfolio_stats(x)[0] - tr},
               {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    
    opt_ef = sco.minimize(min_volatility, initial_wts, method='SLSQP', bounds=bnds, constraints=ef_cons)
    
    tvols.append(opt_ef['fun'])

targetvols = np.array(tvols)

In [52]:
# Create EF Dataframe for plotting
efport = pd.DataFrame({
    'targetrets' : np.around(100*targetrets,2),
    'targetvols': np.around(100*targetvols,2),
    'targetsharpe': np.around(targetrets/targetvols,2)
})

efport.head()

,targetrets,targetvols,targetsharpe
0,15.00,30.18,0.50
1,15.08,30.18,0.50
2,15.15,30.18,0.50
3,15.23,30.18,0.50
4,15.30,30.18,0.51


In [53]:
# Plot efficient frontier portfolio
fig = px.scatter(
    efport, x='targetvols', y='targetrets',  color='targetsharpe',
    labels={'targetrets': 'Expected Return', 'targetvols': 'Expected Volatility','targetsharpe': 'Sharpe Ratio'},
    title="Efficient Frontier Portfolio"
     ).update_traces(mode='markers', marker=dict(symbol='cross'))


# Plot maximum sharpe portfolio
fig.add_scatter(
    mode='markers',
    x=[100*portfolio_stats(opt_sharpe['x'])[1]], 
    y=[100*portfolio_stats(opt_sharpe['x'])[0]],
    marker=dict(color='red', size=20, symbol='star'),
    name = 'Max Sharpe'
).update(layout_showlegend=False)

# Plot minimum variance portfolio
fig.add_scatter(
    mode='markers',
    x=[100*portfolio_stats(opt_var['x'])[1]], 
    y=[100*portfolio_stats(opt_var['x'])[0]],
    marker=dict(color='green', size=20, symbol='star'),
    name = 'Min Variance'
).update(layout_showlegend=False)

# Show spikes
fig.update_xaxes(showspikes=True)
fig.update_yaxes(showspikes=True)
fig.show()